In [2]:
import os
import torch
import random
import numpy as np
from torch.utils.data import DataLoader
from torch import nn

from pose.dataset import pose_dataset
from pose.utils import collate_fn, geodesic_distance, relative_pose_error
from pose.model0606 import MoCoPE
from pose.animator import Animator

/home2/mingxintan/envs/tmx_vim/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CUDA_LAUNCH_BLOCKING=1
num_sample = 500
batch_mkpts0 = torch.rand(1, num_sample, 2).to('cuda:1')
batch_mkpts1 = torch.rand(1, num_sample, 2).to('cuda:1')
batch_img0 = torch.rand(1, 3, 224, 224).to('cuda:1')
batch_img1 = torch.rand(1, 3, 224, 224).to('cuda:1')

In [4]:
type(batch_mkpts0)

torch.Tensor

In [5]:
model = MoCoPE(num_sample=num_sample, vim_model_type='small').to('cuda:1')

/home2/mingxintan/envs/tmx_vim/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
t, r = model(batch_mkpts0, batch_mkpts1, batch_img0, batch_img1)

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
t.shape, r.shape

: 

In [ ]:
gt_t = torch.rand(8, 3)
gt_rot = torch.rand(8, 3, 3)

In [ ]:
L2 = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)

In [ ]:
t_loss = L2(gt_t, t)
rot_loss = geodesic_distance(gt_rot, r)
loss = t_loss + rot_loss

In [ ]:
optimizer.zero_grad()
loss.backward()
optimizer.step()
loss